In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("Hotel_Reviews.csv")
print(f"Dataset has {df.shape[0]} rows and {df.shape[1]} columns")

Dataset has 515738 rows and 17 columns


In [3]:
#Separating the various tags into independent columns so that we can get maximum information and can use these features in modelling
new = df["Tags"].str.split(",", n = -1, expand = True)

In [4]:
#Cleaning up the separated 'Tags' dataframe first, will then merge it back to our dataset

#Naming the new column names for easier reference
new = new.set_axis(['Tag1', 'Tag2', 'Tag3', 'Tag4', 'Tag5', 'Tag6'], axis=1)

In [5]:
#Cleaning column contents

#Removing unwanted special characters, such as ', [, and ]
for column in new:
    new[column] = new[column].str.replace("'", "")
for column in new:
    new[column] = new[column].str.replace("[", "")
for column in new:
    new[column] = new[column].str.replace("]", "")
    
#Removing unwanted leading and trailing spaces within each column
#new['Tag1'] = new['Tag1'].str.strip()
for column in new:
    new[column] = new[column].str.strip()

In [6]:
tag1leisure = new.index[new['Tag1'] == 'Leisure trip'].tolist()

#We will have to remove rows from the original df systematically
#First we keep only leisure trip rows, and only then remove non-child travellers from the remaining rows 
indexestokeep = tag1leisure
len(indexestokeep)

416672

In [7]:
#Checking what information is present in each tag
new.Tag1.unique() #Type of trip
new.Tag2.unique() #Type of trip (Overlap with Tag1) & Room Type
new.Tag3.unique() #Type of Room (Overlap with Tag2)
new.Tag4.unique() #Type of Room (Overlap with Tag2 & Tag3) & Length of Stay
new.Tag5.unique() #Length of Stay (Overlap with Tag4)
new.Tag6.unique() #Submitted from a mobile device

#There's a considerable amount of overlap in the information the tags are offering, and this will be particularly difficult to clean
#We should aim to find some pattern which standardises the information offered, as a lot of the information offered is 'off' by 1 column

#Checking to see the frequency of each category in the columns
new['Tag1'].value_counts()
#Clearly, over 498k rows have 'leisure' or 'business' type of travel stated, and these are the only 2 categories not included in Tag2, suggesting that rows without this information 'Start from Tag1', and rows with this additional information 'Start from Tag2'
#Thus, keeping or removing rows with these values is the key to standardising the dataset

#The value counts shows that 498k rows have this demarkation, and rows that don't have them are in the minority (17k)
#Thus, we will keep the rows with this distinction, and drop the ones that don't have this

new4 = new.loc[indexestokeep]

#Resetting index to ensure we get the accurate indexes for filtering by traveller_category == family
new4 = new4.reset_index(drop=True)

In [8]:
#Repeating the filtering process to filter by travller_category == family
tag2familieswithyoungkids = new4.index[new4['Tag2'] == 'Family with young children'].tolist()
tag2familieswitholdkids = new4.index[new4['Tag2'] == 'Family with older children'].tolist()

indexestokeep1 = tag2familieswithyoungkids + tag2familieswitholdkids
new4 = new4.loc[indexestokeep1]

new4.Tag1.unique() #Type of trip
new4.Tag2.unique() #Who's taking the trip (Travellers type)
new4.Tag3.unique().tolist() #Room Type - the tolist allowed me to view all the unique elements at once
new4.Tag4.unique() #Length of stay
new4.Tag5.unique() #Whether or not submitted from mobile device
new4.Tag6.unique() #None

new4 = new4.reset_index(drop=True)

In [9]:
new4.shape #(499420, 6) <-- This shows that only the relevant rows were kept

#We also need to remove these rows from the original dataframe to ensure that once we clean up the tags and merge the tags df back to the original, the tags are assigned to the appropriate rows
df = df.loc[indexestokeep]
df = df.reset_index(drop=True)

df = df.loc[indexestokeep1]
df = df.reset_index(drop=True)

df.shape #(80946, 17)

(80946, 17)

In [10]:
#With unwanted characters removed, we can create dummy variables for the various categories listed here, and then merge this back to the original dataframe

#Renaming the columns to reflect the information they capture
new4['Tag1'].unique()
new4 = new4.rename(columns = {'Tag1': 'Travel_Type'})

new4['Tag2'].unique()
new4 = new4.rename(columns = {'Tag2': 'Traveller_Category'})

new4['Tag3'].unique()
new4 = new4.rename(columns = {'Tag3': 'Room_Type'})

new4['Tag4'].unique()
new4 = new4.rename(columns = {'Tag4': 'Stay_Length'})

new4['Tag5'].unique()
new4 = new4.rename(columns = {'Tag5': 'Submitted_From_Mobile'})

In [11]:
#Narrowing down the dataset to leisure trips by families with children (both old and young)
new4 = new4[new4.Travel_Type == 'Leisure trip']

new4 = new4[(new4.Traveller_Category == 'Family with young children') | (new4.Traveller_Category == 'Family with older children')]
new4_dict = new4['Room_Type'].value_counts(normalize = True).to_dict() #Filter out values less than 10

new4['Value_Counts_Percent'] = new4['Room_Type']
new4['Value_Counts_Percent'] = new4['Value_Counts_Percent'].map(new4_dict)

#Using only those categories that represent at least 1% of the overall dataset
new4 = new4[new4.Value_Counts_Percent > 0.01]

In [12]:
#Repeating the filtering process to filter by the value_counts threshold
new4 = new4.reset_index(drop=True)

requiredvaluecounts = new4.index[new4['Value_Counts_Percent'] > 0.01].tolist()
new4 = new4.loc[requiredvaluecounts]

#Similarly cleaning the original df to ensure consistency
df = df.loc[requiredvaluecounts]
df = df.reset_index(drop=True)

In [13]:
#Further cleaning up the 'new4' dataset so that processing is easy, after which we can easily merge it back to the original df

#Categorising Submitted_From_Mobile into (1,0) classification
new4['Submitted_From_Mobile'] = new4['Submitted_From_Mobile'].replace("Submitted from a mobile device", 1)
new4['Submitted_From_Mobile'] = new4['Submitted_From_Mobile'].fillna(0)
new4

#Removing Tag6 as it is completely empty
new4 = new4.drop('Tag6', axis = 1)

In [14]:
#Converting the categorical 'Stay_Length' column into continuous
new4['Stay_Length'] = new4.Stay_Length.str.extract('(\d+)')
new4.dtypes

#The data-type is still string, and we need to convert this to numeric
new4.Stay_Length = pd.to_numeric(new4.Stay_Length, errors = 'coerce')

In [ ]:
#Getting dummies for each of the variables in new4 to allow ML models to process the data
#We need to get dummies for only 3 columns - Travel_Type, Traveller_Category, and Room_Type

#Will use one-hot encoding
ohe_Travel_Type = pd.get_dummies(new4['Travel_Type'])
ohe_Traveller_Category = pd.get_dummies(new4['Traveller_Category'])
ohe_Room_Type = pd.get_dummies(new4['Room_Type'])

#We can now merge the one_hot_encoded columns into new4 and drop the original columns
new4 = new4.join([ohe_Travel_Type, ohe_Traveller_Category, ohe_Room_Type])
new4.drop(['Travel_Type', 'Traveller_Category', 'Room_Type'], axis = 1, inplace = True)

In [15]:
#We will now merge the cleaned Tags to the original dataset and drop the original 'Tags' column
df.drop('Tags', axis = 1, inplace = True)
df = df.join(new4)

In [18]:
df.to_csv("Exploration.csv", index = False)

In [ ]:
#We can now proceed to further clean the dataset as a whole

df.isna().sum()
#Latitude & Longitude are the only columns with na values

In [ ]:
df.drop(['Additional_Number_of_Scoring', 'Review_Date', 'days_since_review', 'lat', 'lng'], axis =1, inplace = True)

In [ ]:
# New column to consolidate the negative and positive reviews

df["Review"] = df["Negative_Review"] + df["Positive_Review"]
df.drop(['Negative_Review', 'Positive_Review'], axis =1, inplace = True)

In [ ]:
# Remove 'No Negative' & 'No Positive' text from column bececause user did not give negative and positive reviews

df["Review"] = df["Review"].apply(lambda x: x.replace("No Negative", "").replace("No Positive", ""))

In [ ]:
# Tokenisation process and text cleaning

import nltk
from nltk.corpus import wordnet

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)

# clean text data
df["Review_Clean"] = df["Review"].apply(lambda x: clean_text(x))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
docs=df['Review_Clean'].tolist()
#create a vocabulary of words, 
#ignore words that appear in 85% of documents, 
#eliminate stop words
cv=CountVectorizer(max_df=0.95)
word_count_vector=cv.fit_transform(docs)
word_count_vector

#Now, let's look at 10 words from our vocabulary. Sweet, these are mostly crime and comedy related.
list(cv.vocabulary_.keys())[:100]

In [ ]:
## Important keywords extraction using tfidf

from sklearn.feature_extraction.text import TfidfTransformer
# tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
                stop_words='english',
                use_idf=True,
                lowercase=True,
                smooth_idf=True) 
# tfidf_transformer = vectorizer.fit_transform(df.Review_Clean)


# print(df.Review_Clean)
# vector = cv.transform(df.Review_Clean[1])
tfidf_vector = vectorizer.fit_transform(df.Review_Clean)  # tfidf_transformer.transform(vector)
coo_matrix = tfidf_vector.tocoo()
tuples = zip(coo_matrix.col, coo_matrix.data)
sorted_tuple = sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
scores = pd.DataFrame([(cv.get_feature_names()[i[0]],i[1]) for i in sorted_tuple])

In [ ]:
scores = scores.rename(columns = {0: 'word', 1: 'score'})

In [ ]:
scores.word.nunique() #18041

In [ ]:
#Taking the average score for each word
scores_mean = scores.groupby('word').mean()
scores_mean = scores_mean.reset_index()
words = scores_mean.word.tolist()
scores = scores_mean.score.tolist()
scores_mean

In [ ]:
#Correcting misspelt words identified and scored by tfidf
from autocorrect import Speller
spell = Speller(lang = 'en')

corrected_words = []

for each in words:
    each = spell(each)
    corrected_words.append(each)

#Appending the corrected words to the df with the scores as a separate column
scores_mean = scores_mean.assign(Corrected_Words = corrected_words)

#True and False were mis-identified and their correct spelling corrupted. Restoring their original spellings in lowercase 
scores_mean.Corrected_Words.replace("FAS", "false", inplace = True)
scores_mean.Corrected_Words.replace("RE", "true", inplace = True)

In [ ]:
#Taking the average score for each corrected word. As several words post-correction would be identical but with their earlier scores (Ex: Worlde --> world)  
scores_mean = scores_mean.groupby('Corrected_Words').mean()
scores_mean = scores_mean.reset_index()
words = scores_mean.Corrected_Words.tolist()
score = scores_mean.score.tolist()
scores_dict = dict(zip(words, score))

In [ ]:
#Cleaning out problematic values

#Starting with na values for cleaned review which cannot have a tfidf score attached to them
df_new = df[df['Review_Clean'].notnull()]

#Also editing out row 871, which is 'missing'(Despite resetting index) and causes problems in tdidf score computation for each individual review
df.new1 = df_new[0:870]
df.new2 = df_new[872:38870]
frames = [df.new1, df.new2]
df_final = pd.concat(frames)

In [ ]:
#Running a for-loop to calculate the tfidf scores for each review (row), which is the sum of the tfidf scores of the individual words used to compile the review

tfidf_score = []
score = 0
count = 0
for each_review in df_final['Review_Clean']:
    words_in_review = each_review.split()
    for each_word in words_in_review:
        if each_word in scores_dict:
            score += scores_dict[each_word]
        else:
            pass
    tfidf_score.append(score)
    score = 0
    count += 1

In [ ]:
df_final = df_final.assign(tfidf_score = tfidf_score)
#df_final.drop('Unnamed: 0', axis = 1, inplace = True) <-- Got an extra unwanted column and removed it
df_final.to_csv("df_final.csv") #This is the final, cleaned dataset